<a href="https://colab.research.google.com/github/spe301/AI-generated-AI/blob/main/AI_generated_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Potosnail==0.0.7

In [2]:
import pandas as pd
import numpy as np
from potosnail import MachineLearning, DeepLearning, DataHelper, Evaluater, Algorithms, Wrappers

ml = MachineLearning()
dl = DeepLearning()
dh = DataHelper()
ev = Evaluater()
al = Algorithms()
wr = Wrappers()

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
Text_Classification_Results = pd.read_csv(r'https://raw.githubusercontent.com/spe301/AI-generated-AI/main/Data/Text_Classification_Results.csv').drop(['Unnamed: 0'], axis='columns')
Image_Classification_Results = pd.read_csv(r'https://raw.githubusercontent.com/spe301/AI-generated-AI/main/Data/Image_Classification_Results.csv').drop(['Unnamed: 0'], axis='columns')
Tabular_Classification_Results = pd.read_csv(r'https://raw.githubusercontent.com/spe301/AI-generated-AI/main/Data/Tabular_Classification_Results.csv').drop(['Unnamed: 0'], axis='columns')
Tabular_Regression_Results = pd.read_csv(r'https://raw.githubusercontent.com/spe301/AI-generated-AI/main/Data/Tabular_Regression_Results.csv').drop(['Unnamed: 0'], axis='columns')

def Preprocess(df):
  df = df.fillna('None')
  cols = list(df.columns) 
  to_drop = []
  ohe = []
  for col in cols:
    if type(df[col][0]) == str:
      if len(np.unique(df[col])) == 1:
        to_drop.append(col)
      else:
        ohe.append(col)
    if type(df[col][0]) == np.bool_:
      boolys = []
      for i in range(len(df[col])):
        if df[col][i] == True:
          boolys.append(1)
        if df[col][i] == False:
          boolys.append(0)
      df[col] = boolys
  df2 = df.drop(to_drop, axis='columns')
  try:
    df3 = pd.get_dummies(df[ohe])
  except:
    return df2
  return pd.concat([df2.drop(ohe, axis='columns'), df3], axis=1)

In [4]:
df1 = Preprocess(Text_Classification_Results)
df2 = Preprocess(Image_Classification_Results)
df3 = Preprocess(Tabular_Classification_Results)
df4 = Preprocess(Tabular_Regression_Results)

In [8]:
df4

,nodes,stacking,dropout,nlayers,closer,epochs,batch_size,loss,val_loss,len_dataset,n_features,loss_func_MAE,loss_func_MSE
0,32,0,0,4,0,9,32,73.553352,72.291702,506,13,0,1
1,32,0,0,4,0,17,32,60.416046,116.589890,506,13,1,0
2,64,0,0,4,0,15,32,68.376144,50.315472,506,13,0,1
3,64,0,0,4,0,9,32,65.230408,59.369423,506,13,1,0
4,128,0,0,4,0,15,32,56.645454,52.476025,506,13,0,1
5,128,0,0,4,0,50,32,639.196106,276.652618,506,13,1,0
